#### Parsing environment variable values

For most simple field types (such as `int`, `float`, `str`, etc.), the environment variable value is parsed the same way it would be if passed directly to the initialiser (as a string).

Complex types like `list`, `set`, `dict`, and sub-models are populated from the environment by treating the environment variable's value as a JSON-encoded string.

Another way to populate nested complex variables is to configure your model with the `env_nested_delimiter` config setting, then use an env variable with a name pointing to the nested module fields. What it does is simply explodes your variable into nested models or dicts. So if you define a variable `FOO__BAR__BAZ=123` it will convert it into `FOO={"BAR": {"BAZ": 123}}` If you have multiple variables with the same structure they will be merged.

In [1]:
import os
from typing import Any, List
from pydantic import BaseModel, BaseSettings

In [2]:
class DeepSubModel(BaseModel):
    v4: str

In [3]:
class SubModel(BaseModel):
    v1: str
    v2: bytes
    v3: int
    deep: DeepSubModel

In [4]:
class Settings(BaseSettings):
    v0: str
    sub_model: SubModel

    class Config:
        env_nested_delimiter = "__"

In [5]:
os.environ["V0"] = "0"
os.environ["SUB_MODEL"] = '{"v1": "json-1", "v2": "json-2"}'
os.environ["SUB_MODEL__V2"] = "nested-2"
os.environ["SUB_MODEL__V3"] = "3"
os.environ["SUB_MODEL__DEEP__V4"] = "v4"

In [6]:
print(Settings().dict())

{'v0': '0', 'sub_model': {'v1': 'json-1', 'v2': b'nested-2', 'v3': 3, 'deep': {'v4': 'v4'}}}


`env_nested_delimiter` can be configured via the `Config` class as shown above, or via the `_env_nested_delimiter` keyword argument on instantiation.

JSON is only parsed in top-level fields, if you need to parse JSON in sub-models, you will need to implement validators on those models.

Nested environment variables take precedence over the top-level environment variable JSON (e.g. in the example above, `SUB_MODEL__V2` trumps `SUB_MODEL`).

You may also populate a complex type by providing your own parsing function to the `parse_env_var` classmethod in the Config object.

In [7]:
class NestedSettings(BaseSettings):
    numbers: List[int]

    class Config:
        @classmethod
        def parse_env_var(cls, field_name: str, raw_val: str) -> Any:
            if field_name == "numbers":
                return [int(x) for x in raw_val.split(",")]
            return cls.json_loads(raw_val)

In [8]:
os.environ["numbers"] = "1,2,3"
print(NestedSettings().dict())

{'numbers': [1, 2, 3]}
